「アイスを買ったから離脱」ではなく、「間隔が空き始めた後にアイスを買ったから離脱」という文脈を捉えます。

仮説: 離脱するユーザーは、突然止めるのではなく、購入間隔が徐々に広がる（フェードアウトする）。

特徴量の作り方（イベント定義）: カテゴリ名の後ろに、前回購入からの経過日数を離散化して付与します。

ビール_Short (前回から1週間以内)

ビール_Medium (前回から2週間以内)

ビール_Long (前回から1ヶ月以上)

検知したいパターン:

[ビール_Short] → [ビール_Medium] → [ビール_Long] → [離脱]

※ これにより、「頻繁に買っていた人が、徐々に疎遠になっている」という**「勢いの減衰」**自体をパターンとして学習できます。

In [1]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. ダミーデータの作成 (user_id, category, timestamp)
# ---------------------------------------------------------
data = [
    # ユーザー1: 短い間隔で繰り返し購入 (ロイヤル層)
    {'user_id': 1, 'category': 'ビール', 'timestamp': '2023-01-01'},
    {'user_id': 1, 'category': 'おつまみ', 'timestamp': '2023-01-01'}, # 同時購入
    {'user_id': 1, 'category': 'ビール', 'timestamp': '2023-01-05'}, # 4日後 (Short)
    {'user_id': 1, 'category': 'ビール', 'timestamp': '2023-01-05'}, # 同一バスケットの重複 (除去対象)
    {'user_id': 1, 'category': 'ビール', 'timestamp': '2023-01-10'}, # 5日後 (Short)
    
    # ユーザー2: 徐々に間隔が空いていく (離脱予備軍)
    {'user_id': 2, 'category': 'ビール', 'timestamp': '2023-02-01'},
    {'user_id': 2, 'category': 'ビール', 'timestamp': '2023-02-05'}, # 4日後 (Short)
    {'user_id': 2, 'category': 'ビール', 'timestamp': '2023-02-20'}, # 15日後 (Medium)
    {'user_id': 2, 'category': 'ビール', 'timestamp': '2023-04-01'}, # 40日後 (Long)
]

df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ---------------------------------------------------------
# 2. 前処理: 重複排除とソート
# ---------------------------------------------------------
# 同一レシート(同じ日時)で複数行ある場合、間隔計算のために1つに絞る
# ※個数も特徴量にしたい場合は別途集計が必要ですが、今回は「間隔」を見るため削除
df_unique = df.drop_duplicates(subset=['user_id', 'timestamp', 'category']).copy()

# ユーザーIDと日時でソート (必須)
df_unique = df_unique.sort_values(by=['user_id', 'timestamp'])

# ---------------------------------------------------------
# 3. 間隔(Gap)の計算ロジック
# ---------------------------------------------------------
def categorize_gap(row):
    # ビール以外は何もしない
    if row['category'] != 'ビール':
        return row['category']
    
    # 初回購入の場合 (NaN)
    if pd.isna(row['days_diff']):
        return 'ビール_First'
    
    # 間隔によるラベリング定義
    gap = row['days_diff']
    if gap <= 7:
        return 'ビール_Short'   # 1週間以内
    elif gap <= 30:
        return 'ビール_Medium'  # 1ヶ月以内
    else:
        return 'ビール_Long'    # 1ヶ月以上

# ---------------------------------------------------------
# 4. 特徴量生成の実行
# ---------------------------------------------------------

# ビールの行だけを抜き出して、前回購入時との差分を計算
beer_mask = df_unique['category'] == 'ビール'

# user_idごとにグループ化し、timestampの差分(diff)を取る
# shift(1)で1つ前の時間を取得し、現在との差を計算
df_unique.loc[beer_mask, 'prev_ts'] = df_unique.loc[beer_mask].groupby('user_id')['timestamp'].shift(1)
df_unique.loc[beer_mask, 'days_diff'] = (df_unique.loc[beer_mask, 'timestamp'] - df_unique.loc[beer_mask, 'prev_ts']).dt.days

# ラベリング関数を適用して新しいカテゴリ名を作成
df_unique['new_category'] = df_unique.apply(categorize_gap, axis=1)

# ---------------------------------------------------------
# 5. seq2pat用のリスト形式に変換
# ---------------------------------------------------------
# ユーザーごとに時系列順にリスト化
seq_data = df_unique.groupby('user_id')['new_category'].apply(list).reset_index()

print("--- 作成されたデータフレーム (確認用) ---")
print(df_unique[['user_id', 'timestamp', 'category', 'days_diff', 'new_category']])

print("\n--- seq2pat入力用シーケンス ---")
print(seq_data)

--- 作成されたデータフレーム (確認用) ---
   user_id  timestamp category  days_diff new_category
0        1 2023-01-01      ビール        NaN    ビール_First
1        1 2023-01-01     おつまみ        NaN         おつまみ
2        1 2023-01-05      ビール        4.0    ビール_Short
4        1 2023-01-10      ビール        5.0    ビール_Short
5        2 2023-02-01      ビール        NaN    ビール_First
6        2 2023-02-05      ビール        4.0    ビール_Short
7        2 2023-02-20      ビール       15.0   ビール_Medium
8        2 2023-04-01      ビール       40.0     ビール_Long

--- seq2pat入力用シーケンス ---
   user_id                                  new_category
0        1       [ビール_First, おつまみ, ビール_Short, ビール_Short]
1        2  [ビール_First, ビール_Short, ビール_Medium, ビール_Long]


「同時購入（Basket Context）」の変化
「ビールと一緒に何を買っていたか」の変化は、ライフスタイルの変化を強く反映します。

仮説: 「夕食の食材と一緒に買う（晩酌需要）」から、「スナック菓子と一緒に買う（嗜好品需要）」へ変化し、最終的に「買わなくなる」という流れがある。またはその逆。

特徴量の作り方: ビールの購入カテゴリそのものではなく、「バスケットの主要カテゴリ」をイベントにします。

Dinner_Basket_with_Beer: ビール + {精肉, 鮮魚, 惣菜}

Snack_Basket_with_Beer: ビール + {菓子, 珍味}

Solo_Beer: ビールのみ

検知したいパターン:

[Dinner_Basket_with_Beer] → [Solo_Beer] → [離脱]

（食事と一緒に楽しまなくなり、単なるアルコール摂取になり、やがて止めるパターン）

この「同時購入（Basket Context）」によるイベント定義は、ユーザーの生活スタイルの変化（例：自炊派から個食・孤食への変化など）を捉えるのに非常に有効です。

実装のポイントは、**「user_id と timestamp でグルーピング（＝バスケット化）し、その中身（categoryの集合）を判定ロジックにかける」**という点です。

以下に、Pandasを使ってこの「バスケット内容に基づくイベントラベル」を作成するコードを作成しました。

実装のポイント
優先順位（Priority）: 実際の買い物では「ビール＋惣菜＋スナック」のように混ざることがあります。今回は「食事（Dinner）」を最も強い文脈とし、Dinner > Snack > Solo の順で優先度をつけて判定します。

集合演算（Set Operation）: isin や any を使うと高速に判定できます。

さらなる工夫（応用編）
このコードで基本的な「同時購入コンテキスト」は作れますが、先ほどの「間隔（Gap）」の話と組み合わせるとさらに強力になります。

例えば、イベント名を結合して：

Dinner_Beer_Short（頻繁に晩酌している＝安定）

Solo_Beer_Long（久しぶりにビールだけ買った＝危険信号？）

のように、「コンテキスト × 間隔」 の複合イベントを作ると、seq2pat がよりリッチなパターン（文脈の変化＋頻度の変化）を学習できるようになります。もしデータ量（イベントの種類）が増えすぎてスパースになるようであれば、まずは単体で試してみてください。

In [2]:
import pandas as pd

# ---------------------------------------------------------
# 1. カテゴリ定義（ここを自社データに合わせて調整してください）
# ---------------------------------------------------------
TARGET_ITEM = 'ビール'
DINNER_CATS = {'精肉', '鮮魚', '惣菜', '野菜'} # 食事系
SNACK_CATS  = {'菓子', '珍味', 'スナック'}      # つまみ系

# ---------------------------------------------------------
# 2. ダミーデータの作成
# ---------------------------------------------------------
data = [
    # --- バスケットA: ビール + 惣菜 (Dinner文脈) ---
    {'user_id': 1, 'timestamp': '2023-01-01 18:00', 'category': 'ビール'},
    {'user_id': 1, 'timestamp': '2023-01-01 18:00', 'category': '惣菜'},
    {'user_id': 1, 'timestamp': '2023-01-01 18:00', 'category': '牛乳'}, # 関係ないものも混ざる
    
    # --- バスケットB: ビール + 菓子 (Snack文脈) ---
    {'user_id': 1, 'timestamp': '2023-01-05 20:00', 'category': 'ビール'},
    {'user_id': 1, 'timestamp': '2023-01-05 20:00', 'category': '菓子'},
    
    # --- バスケットC: ビールのみ (Solo文脈) ---
    {'user_id': 1, 'timestamp': '2023-01-10 22:00', 'category': 'ビール'},
    
    # --- バスケットD: ビールなし (今回は無視するか、No_Beerとして扱うか) ---
    {'user_id': 1, 'timestamp': '2023-01-15 10:00', 'category': 'パン'},
]

df = pd.DataFrame(data)

# ---------------------------------------------------------
# 3. バスケットごとの集約と判定ロジック
# ---------------------------------------------------------

# user_id と timestamp ごとに、購入カテゴリを「集合(set)」にまとめる
# これにより、1つのレシートが1行のデータになります
basket_df = df.groupby(['user_id', 'timestamp'])['category'].apply(set).reset_index()

def classify_basket(categories):
    """
    バスケットの中身(set)を受け取り、イベント名を返す関数
    """
    # 1. そもそもビールが入っていない場合
    if TARGET_ITEM not in categories:
        return 'No_Beer_Basket' # 必要なければ None を返して後で除外
    
    # 2. 食事系が含まれているか？ (Dinner優先)
    # create intersection to check if any dinner cat exists
    if not categories.isdisjoint(DINNER_CATS):
        return 'Dinner_Basket_with_Beer'
    
    # 3. つまみ系が含まれているか？ (Snack)
    elif not categories.isdisjoint(SNACK_CATS):
        return 'Snack_Basket_with_Beer'
    
    # 4. 上記以外でビールが含まれている (Solo、またはその他日用品のみ)
    else:
        return 'Solo_Beer'

# 判定関数を適用
basket_df['event_label'] = basket_df['category'].apply(classify_basket)

# ビールを含まないバスケットを除外したい場合はここでフィルタリング
# seq2pat用にするため、今回はビールを含むものだけに絞ります
beer_baskets = basket_df[basket_df['event_label'] != 'No_Beer_Basket'].copy()

# 時系列順にソート（念のため）
beer_baskets = beer_baskets.sort_values(by=['user_id', 'timestamp'])

# ---------------------------------------------------------
# 4. seq2pat用のシーケンス作成
# ---------------------------------------------------------
seq_data = beer_baskets.groupby('user_id')['event_label'].apply(list).reset_index()

# 結果確認
print("--- バスケットごとの判定結果 ---")
print(beer_baskets[['user_id', 'timestamp', 'event_label']])

print("\n--- seq2pat入力用シーケンス ---")
print(seq_data)

--- バスケットごとの判定結果 ---
   user_id         timestamp              event_label
0        1  2023-01-01 18:00  Dinner_Basket_with_Beer
1        1  2023-01-05 20:00   Snack_Basket_with_Beer
2        1  2023-01-10 22:00                Solo_Beer

--- seq2pat入力用シーケンス ---
   user_id                                        event_label
0        1  [Dinner_Basket_with_Beer, Snack_Basket_with_Be...


もちろんです。できます。 この「カニバリゼーション（競合への食い合い）」を検証するには、「ビールの有無」と「代替品の有無」の組み合わせ（Matrix） でイベント名を定義するのが最も手っ取り早く、かつ強力です。

単に [ビール] [RTD] とするのではなく、[Beer_with_RTD]（併売） という専用のイベントを作ることで、「浮気期間（併用期間）」を明確にパターンとして検出できるようになります。

以下に、Pandasを使ってこの「競合併売ラベル」を作成する実装コードを作成しました。

実装のポイント
代替品辞書の定義: RTD、炭酸水、ノンアルなど、複数の代替品カテゴリを辞書で管理し、それぞれに対して「Beerのみ」「代替品のみ」「併売（Mix）」を判定します。

優先順位: もし1つのレシートに「RTD」と「炭酸水」が両方入っていた場合、より強力な競合である「RTD」を優先してイベント名にするロジックにしています（変更可能です）。

RTDへのカニバリ離脱:

パターン: [Beer_Only] -> [Mix_Beer_RTD] -> [RTD_Only]

解釈: 「試しにチューハイも買ってみたら美味しかった/安かったので、そっちがメインになった」層。

健康系への離脱:

パターン: [Beer_Only] -> [Soda_Only]

解釈: 「休肝日を作ろうとして、そのままフェードアウトした」層。

踏み止まり（離脱回避）:

パターン: [Mix_Beer_RTD] -> [Beer_Only]

解釈: 「浮気してみたけど、やっぱりビールに戻ってきた」層。

In [3]:
import pandas as pd

# ---------------------------------------------------------
# 1. カテゴリ定義 (ここを自社のカテゴリ名に合わせてください)
# ---------------------------------------------------------
TARGET_ITEM = 'ビール'

# 代替品(Substitutes)の定義。辞書形式で管理すると拡張しやすいです。
# 優先度順（上のキーほど優先される）に並べておくのがコツです。
SUBSTITUTES = {
    'RTD': {'チューハイ', 'サワー', 'ハイボール'},   # アルコール競合
    'NonAlc': {'ノンアルコールビール', '微アル'},     # 休肝日・断酒
    'Soda': {'炭酸水', '強炭酸水'},                 # のどごし代替
}

# ---------------------------------------------------------
# 2. ダミーデータの作成
# ---------------------------------------------------------
data = [
    # ユーザー1: ビール党 -> RTD併用 -> RTDへ完全移行
    {'user_id': 1, 'timestamp': '2023-01-01', 'category': 'ビール'},
    
    {'user_id': 1, 'timestamp': '2023-01-10', 'category': 'ビール'},
    {'user_id': 1, 'timestamp': '2023-01-10', 'category': 'チューハイ'}, # 併売(Mix)発生
    
    {'user_id': 1, 'timestamp': '2023-02-01', 'category': 'チューハイ'}, # 完全移行
    {'user_id': 1, 'timestamp': '2023-02-05', 'category': 'サワー'},
    
    # ユーザー2: ビール -> 炭酸水 (健康志向離脱)
    {'user_id': 2, 'timestamp': '2023-01-01', 'category': 'ビール'},
    {'user_id': 2, 'timestamp': '2023-01-15', 'category': '炭酸水'},
    
    # ユーザー3: 関係ない買い物 (ノイズ)
    {'user_id': 3, 'timestamp': '2023-01-01', 'category': '牛乳'},
]

df = pd.DataFrame(data)

# ---------------------------------------------------------
# 3. バスケットごとの判定ロジック
# ---------------------------------------------------------

# user_id と timestamp ごとにカテゴリを集合(set)にする
basket_df = df.groupby(['user_id', 'timestamp'])['category'].apply(set).reset_index()

def classify_cannibalization(categories):
    """
    バスケットの中身を見て、ビールと代替品の関係をラベル化する
    """
    has_beer = TARGET_ITEM in categories
    
    # 代替品が含まれているかチェック（優先度順）
    detected_sub = None
    for sub_label, sub_cats in SUBSTITUTES.items():
        # 積集合(intersection)が空でなければ、その代替品が含まれている
        if not categories.isdisjoint(sub_cats):
            detected_sub = sub_label
            break # 最も優先度の高い代替品が見つかったらそこで判定終了
    
    # --- パターン定義 ---
    
    # 1. ビールも代替品もある場合 (最重要: 併売/Mix)
    if has_beer and detected_sub:
        return f'Mix_Beer_{detected_sub}' # 例: Mix_Beer_RTD
    
    # 2. ビールのみある場合
    elif has_beer:
        return 'Beer_Only'
    
    # 3. 代替品のみある場合 (完全移行)
    elif detected_sub:
        return f'{detected_sub}_Only' # 例: RTD_Only
    
    # 4. どちらもない (今回の分析対象外)
    else:
        return 'Other_Purchase'

# 関数を適用
basket_df['event_label'] = basket_df['category'].apply(classify_cannibalization)

# seq2pat用には「その他(Other_Purchase)」を除くか、
# あるいは「ビール関連または代替品」だけ抽出する
target_events = basket_df[basket_df['event_label'] != 'Other_Purchase'].copy()

# 時系列ソート
target_events = target_events.sort_values(by=['user_id', 'timestamp'])

# ---------------------------------------------------------
# 4. seq2pat用のシーケンス作成
# ---------------------------------------------------------
seq_data = target_events.groupby('user_id')['event_label'].apply(list).reset_index()

print("--- バスケット判定結果 ---")
print(target_events[['user_id', 'timestamp', 'event_label']])

print("\n--- seq2pat入力用シーケンス ---")
print(seq_data)

--- バスケット判定結果 ---
   user_id   timestamp   event_label
0        1  2023-01-01     Beer_Only
1        1  2023-01-10  Mix_Beer_RTD
2        1  2023-02-01      RTD_Only
3        1  2023-02-05      RTD_Only
4        2  2023-01-01     Beer_Only
5        2  2023-01-15     Soda_Only

--- seq2pat入力用シーケンス ---
   user_id                                    event_label
0        1  [Beer_Only, Mix_Beer_RTD, RTD_Only, RTD_Only]
1        2                         [Beer_Only, Soda_Only]


In [4]:
# 健康意識の高い特定アイテム群
HEALTH_PROTEIN = {'サラダチキン', 'プロテイン', '鶏胸肉', 'ブロッコリー'} # 筋トレ・ダイエット
HEALTH_LIVER   = {'ウコン', 'ヘパリーゼ', 'しじみ', 'オルニチン'}     # 肝臓ケア
HEALTH_TOKUHO  = {'黒烏龍茶', '特茶', 'トマトジュース'}               # 脂肪・血圧

def classify_health_context(categories):
    has_beer = 'ビール' in categories
    
    # 1. 筋トレ・ガチダイエット要素があるか？ (離脱リスク大)
    if not categories.isdisjoint(HEALTH_PROTEIN):
        return 'Beer_with_DietFood' if has_beer else 'DietFood_Only'
        
    # 2. 肝臓ケア要素があるか？ (機能性への移行予兆)
    elif not categories.isdisjoint(HEALTH_LIVER):
        return 'Beer_with_LiverCare' if has_beer else 'LiverCare_Only'
        
    # 3. トクホ系があるか？
    elif not categories.isdisjoint(HEALTH_TOKUHO):
        return 'Beer_with_Tokuho' if has_beer else 'Tokuho_Only'
        
    else:
        return 'Normal'

「今日はたまたまビールだけ買った（Solo）」だけで、「次はちゃんと料理と買う（Dinner）」という**「揺らぎ」**は頻繁に起こります。 単発の Dinner → Solo だけを見て「離脱だ！」と判断するのは早計で、モデルの精度を下げるノイズになりかねません。

この「揺らぎ」と「本質的な離脱（不可逆な変化）」を見分けるためには、単一のイベントではなく、「状態の定着（Persistence）」 または 「複合条件」 で捉える必要があります。

seq2pat でこの問題を解決するための、より実践的な3つのアプローチを提案します。

1. 「文脈」×「間隔（Gap）」の掛け合わせ（最強の組み合わせ）
先ほどの「間隔」のアイデアと組み合わせることで、意味合いが全く変わります。

継続ユーザーのパターン（揺らぎ）:

[Dinner_Short] → [Solo_Short] → [Dinner_Short]

（「今回はつまみナシでいいや」と思ったが、すぐに（Short） 次の購入が来ているので、習慣は維持されている）

離脱ユーザーのパターン（劣化）:

[Dinner_Short] → [Solo_Long] → [Solo_Long]

（食事と一緒に買わなくなった上に、購入間隔も空き始めた。これは「生活からビールが遠のいている」明確なサイン）

解決策: イベント名を単なる Solo ではなく、Solo_Long_Gap のように時間軸とセットにしてください。これで「ただの気まぐれなSolo」と「疎遠になりつつあるSolo」を区別できます。

2. 「連続性（Consecutive）」を制約にする
「1回だけの Solo は無視し、2〜3回続いたら変化とみなす」という考え方です。 seq2pat のマイニング設定や、事前のデータ加工でこれを表現します。

データ加工での工夫（Run-Length Encoding的発想）:

直近3回の買い物がすべて Solo の場合のみ、イベントを Stable_Solo_Mode にする。

それ以外（混ざっている場合）は Mixed_Mode とする。

検知したいパターン:

[Stable_Dinner_Mode] → [Mixed_Mode] → [Stable_Solo_Mode] → [離脱]

これなら「たまたま1回だけ」のノイズは Mixed_Mode に吸収され、「完全に習慣が変わった瞬間」 だけを捉えられます。

3. 「消失（Disappearance）」をイベント化する
「Solo が現れたこと」よりも、「Dinner が消えたこと」 の方が離脱シグナルとして強い場合があります。

特定の行動をしなくなったことを検知するために、「過去N回における Dinner含有率」 を計算し、それをイベント化します。

イベント定義:

High_Quality_User: 直近5回中、3回以上が Dinner/Snack 併売

Low_Quality_User: 直近5回中、併売がほぼゼロ

パターン:

[High_Quality] → [High_Quality] → [Low_Quality]

これなら、1回や2回の Solo が混ざっても High_Quality ステータスは維持されるため、ノイズに強くなります。

推奨する具体的な実装修正案
今の懸念を払拭するには、「1. 文脈 × 間隔」 が最も実装コストが低く、かつ効果的です。 前回のコードの classify_basket の結果に、categorize_gap の結果を結合する形に修正しましょう。

Python
# 概念イメージ
def create_rich_event(row):
    # Context (Dinner, Snack, Solo)
    context = classify_basket(row['categories']) 
    
    # Gap (Short, Medium, Long)
    gap = categorize_gap(row['days_diff'])
    
    # 結合して新しいイベント名にする
    return f"{context}_{gap}" 
こうすると、以下のような強力なシーケンスが生まれます。

健全な揺らぎ: Dinner_Short → Solo_Short → Dinner_Short

(モデルはこれを「離脱しない」と学習します)

危険な兆候: Dinner_Short → Solo_Medium → Solo_Long

(モデルはこれを「離脱する」と学習します)

結論: 単体の Solo はご懸念の通りノイズですが、「間隔が伸びている Solo」 は、ほぼ間違いなく**「興味の喪失」または「他店への流出」**を意味します。この組み合わせで勝負してみてください。

このコードでは、以下の2人のユーザーの挙動の違いが明確になるように設計しています。

継続ユーザー（User 1）:

Dinner → Solo（ただし間隔は短い）→ Dinner

解釈：たまにつまみを買わない日があっても、すぐにまた来店しているので「習慣」は継続している。

離脱ユーザー（User 2）:

Dinner → Solo（そして間隔が長い）→ Solo（さらに間隔が長い）

解釈：つまみを買わなくなった上に、来店頻度も落ちている。「興味の喪失」が起きている。

In [5]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. カテゴリ定義
# ---------------------------------------------------------
TARGET_ITEM = 'ビール'
DINNER_CATS = {'精肉', '鮮魚', '惣菜', '野菜'} # 食事系 (最強の文脈)
SNACK_CATS  = {'菓子', '珍味', 'スナック'}      # つまみ系 (中程度の文脈)

# ---------------------------------------------------------
# 2. ダミーデータの作成
# ---------------------------------------------------------
data = [
    # --- ユーザー1: 継続・安定型 (揺らぎはあるが間隔は短い) ---
    # 1/1: ビール + 惣菜 (Dinner)
    {'user_id': 1, 'timestamp': '2023-01-01', 'category': 'ビール'},
    {'user_id': 1, 'timestamp': '2023-01-01', 'category': '惣菜'},
    
    # 1/3: ビールのみ (Solo) -> でも2日後なので Short
    {'user_id': 1, 'timestamp': '2023-01-03', 'category': 'ビール'},
    
    # 1/5: ビール + 菓子 (Snack) -> 2日後 Short
    {'user_id': 1, 'timestamp': '2023-01-05', 'category': 'ビール'},
    {'user_id': 1, 'timestamp': '2023-01-05', 'category': '菓子'},

    # 1/8: ビール + 肉 (Dinner) -> 3日後 Short (戻ってきた!)
    {'user_id': 1, 'timestamp': '2023-01-08', 'category': 'ビール'},
    {'user_id': 1, 'timestamp': '2023-01-08', 'category': '精肉'},


    # --- ユーザー2: 離脱・フェードアウト型 (文脈も頻度も劣化) ---
    # 1/1: ビール + 惣菜 (Dinner)
    {'user_id': 2, 'timestamp': '2023-01-01', 'category': 'ビール'},
    {'user_id': 2, 'timestamp': '2023-01-01', 'category': '惣菜'},

    # 1/15: ビールのみ (Solo) -> 14日後 (Medium Gap) -> 黄信号
    {'user_id': 2, 'timestamp': '2023-01-15', 'category': 'ビール'},

    # 2/20: ビールのみ (Solo) -> 35日後 (Long Gap) -> 赤信号 (完全離脱コース)
    {'user_id': 2, 'timestamp': '2023-02-20', 'category': 'ビール'},
]

df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ---------------------------------------------------------
# 3. バスケット生成 & フィルタリング
# ---------------------------------------------------------
# user_id と timestamp ごとにカテゴリを集合(set)にする
basket_df = df.groupby(['user_id', 'timestamp'])['category'].apply(set).reset_index()

# 今回は「ビールの購買行動の変化」を見たいので、ビールを含まない買い物は除外するか、
# あるいは間隔計算に含めない方針をとります。(ここではビールを含むバスケットのみ抽出)
beer_baskets = basket_df[basket_df['category'].apply(lambda x: TARGET_ITEM in x)].copy()

# 時系列ソート
beer_baskets = beer_baskets.sort_values(by=['user_id', 'timestamp'])

# ---------------------------------------------------------
# 4. ロジック関数定義 (Context & Gap)
# ---------------------------------------------------------

# A. 文脈判定 (Context)
def get_context(categories):
    # 優先順位: Dinner > Snack > Solo
    if not categories.isdisjoint(DINNER_CATS):
        return 'Dinner'
    elif not categories.isdisjoint(SNACK_CATS):
        return 'Snack'
    else:
        return 'Solo'

# B. 間隔判定 (Gap)
def get_gap_label(days_diff):
    if pd.isna(days_diff):
        return 'First'   # 初回
    elif days_diff <= 7:
        return 'Short'   # 1週間以内 (習慣的)
    elif days_diff <= 30:
        return 'Medium'  # 1ヶ月以内 (やや空き)
    else:
        return 'Long'    # 1ヶ月以上 (久しぶり・離脱懸念)

# ---------------------------------------------------------
# 5. 特徴量生成実行
# ---------------------------------------------------------

# (1) Contextの適用
beer_baskets['context_label'] = beer_baskets['category'].apply(get_context)

# (2) Gapの計算
# ユーザーごとに前回のタイムスタンプを取得
beer_baskets['prev_ts'] = beer_baskets.groupby('user_id')['timestamp'].shift(1)
beer_baskets['diff'] = (beer_baskets['timestamp'] - beer_baskets['prev_ts']).dt.days

# (3) Gapラベルの適用
beer_baskets['gap_label'] = beer_baskets['diff'].apply(get_gap_label)

# (4) 最終イベント名の結合 (最強の組み合わせ)
beer_baskets['event_combined'] = beer_baskets['context_label'] + '_' + beer_baskets['gap_label']

# ---------------------------------------------------------
# 6. seq2pat用シーケンス作成
# ---------------------------------------------------------
seq_data = beer_baskets.groupby('user_id')['event_combined'].apply(list).reset_index()

print("--- 詳細データ確認 (Debug) ---")
print(beer_baskets[['user_id', 'timestamp', 'diff', 'context_label', 'gap_label', 'event_combined']])

print("\n--- seq2pat入力用シーケンス ---")
print(seq_data)

--- 詳細データ確認 (Debug) ---
   user_id  timestamp  diff context_label gap_label event_combined
0        1 2023-01-01   NaN        Dinner     First   Dinner_First
1        1 2023-01-03   2.0          Solo     Short     Solo_Short
2        1 2023-01-05   2.0         Snack     Short    Snack_Short
3        1 2023-01-08   3.0        Dinner     Short   Dinner_Short
4        2 2023-01-01   NaN        Dinner     First   Dinner_First
5        2 2023-01-15  14.0          Solo    Medium    Solo_Medium
6        2 2023-02-20  36.0          Solo      Long      Solo_Long

--- seq2pat入力用シーケンス ---
   user_id                                     event_combined
0        1  [Dinner_First, Solo_Short, Snack_Short, Dinner...
1        2             [Dinner_First, Solo_Medium, Solo_Long]
